In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
# Cargo los datos de las películas\n",
df = pd.read_csv('movies_dataset_final.csv', sep = ',')

In [ ]:
# Obtengo una lista de todos los títulos de las películas
titulos = df['title'].fillna('').tolist()

# Concateno todos los títulos en una sola cadena de texto
texto = ' '.join(titulos)

In [ ]:
# Creo un objeto WordCloud con las configuraciones deseadas
wordcloud = WordCloud(width=800, height=400, max_words=100, background_color='black').generate(texto)

In [ ]:
# Importo estas dos técnica de la librería sklearn que son usadas para modelos de recomendación de películas, la primera permite convertir texto
# de películas en características numéricas, los algoritmos lo utlizan para calcular similitud entre películas y hacer recomendacines más precisas.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  # mide la similitud de las películas utilizando las similitud del coseno entre sus vectores


In [ ]:
df = df[['overview', 'popularity','title', 'generos_name']]

In [ ]:
df.info()

In [ ]:
df.head(1)

In [ ]:
df.to_csv('movies_ml1.csv')

In [ ]:
df_ML1 = pd.read_csv('movies_ml1.csv')

In [ ]:
df_ML1.shape

In [ ]:

df_ML1.info()

In [ ]:
# Eliminamos las filas que contengan nulos.
df_ML1.dropna(inplace=True)

In [ ]:
# El dataset es muy grande, por lo tanto para que no nos genere error, lo achicamos
df_subset = df_ML1.head(5000)

In [ ]:
# Guardamos este subconjunto en un nuevo csv, que será nuestro dataset para el modelo a crear.
df_subset.to_csv('movies_ML_p1.csv')

In [ ]:
# Leemos el nuevo csv
movies_nuevo = pd.read_csv("movies_ML_p1.csv")

In [ ]:
movies_nuevo['popularity'] = movies_nuevo['popularity'].astype(float)

In [ ]:
movies_nuevo.info()

In [ ]:
# Eliminar la columna "Unnamed: 0"
movies_ML = movies_nuevo.drop("Unnamed: 0", axis=1)

In [ ]:
movies_ML.head(1)

In [ ]:
movies_ML = movies_ML.drop("Unnamed: 0.1", axis=1)

In [ ]:
movies_ML.head(1)

In [ ]:
# Creo una matriz TF-IDF para el texto del título de las películas
stopwords_custom = ["where","on","the", "at", "in", "of","and"]  # Agrega aquí stopwords personalizados\n",
tfidf = TfidfVectorizer(stop_words=stopwords_custom)
tfidf_matrix = tfidf.fit_transform(movies_ML['title']+' '+ movies_ML['overview'])

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
# Calculamos con linear_kernel la similitud del coseno entre los títulos de las películas
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
def recomendacion(titulo):
    # Verifico si el título está en el DataFrame
    if titulo not in movies_ML['title'].values:
       return f"No se encontró ninguna película con el título '{titulo}'."

    # Obtener el índice de la película con el título dado
    indices = pd.Series(movies_ML.index, index=movies_ML['title']).drop_duplicates()
    idx = indices[titulo]

    # Obtener las puntuaciones de similitud de todas las películas con la película dada
    sim_scores = list(enumerate(cosine_similarities[idx]))

    # Ordenar las películas por puntaje de similitud en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las películas más similares (excluyendo la película dada)
    sim_scores = sim_scores[1:6]  # Obtener las 5 películas más similares
    movie_indices = [x[0] for x in sim_scores]

    # Obtener los títulos de las películas más similares
    respuesta_recomendacion = movies_ML['title'].iloc[movie_indices].tolist()

    return respuesta_recomendacion


In [ ]:
titulo_pelicula = "Nixon"  # Ejemplo
resultado = recomendacion(titulo_pelicula)
print(resultado)